#### Simple Gen AI APP Using Langchain

In [1]:
## Step 0 Initial the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["HUGGINGFACE_API_TOKEN"]=os.getenv("HUGGING_FACE_API_KEY")

In [2]:
## Step 1 Loading the document from the text file ##
from langchain_community.document_loaders import TextLoader, UnstructuredPDFLoader
loader=TextLoader("sampletextfile")
text_documents=loader.load()
text_documents

/Users/parthasarathy.sivakasi/Library/CloudStorage/OneDrive-RackspaceInc/Projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': 'sampletextfile'}, page_content=' Date                   Details                        Merchant .City                  Amount\n   PARTHASARATHY\n\n   24/06/18          WITCO   ( INDIA) PVT LTD              CHENNAI                         2,049.00 \n   29/06/18          S Square Hospitalities                Chennai                         1,218 . 00 \n   29/06/18          HOTEL SAVERA                          CHENNAI                           620 . 00 \n   30/06/18          Ha. thway OERN                        Numb ai                         1,550 . 52 \n   01/07/18          BAGHYAA HOME NEEDS                    CHENNAI                         1,100 . 00\n   01/07/18          CYCL OGENS                            CHENNAI                       13,434.00\n   01/07/18          SPN HOTELS PRIVATE LIM                CHENNAI                         1,800 . 00\n   01/07/18          AMAZON SELLER SERVICE S               MUMS AI                           286 . 11\

In [3]:
# Step 2 Creating the vector store using FAISS and Ollama Embeddings ##
import faiss, langchain_community, ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embeddings=OllamaEmbeddings(model='nomic-embed-text:v1.5')
ollama.embeddings(model='nomic-embed-text:v1.5', prompt='The sky is blue because of rayleigh scattering')


/var/folders/x6/vctnxx216vj_53t96qbwg1r00000gp/T/ipykernel_2768/1399918779.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model='nomic-embed-text:v1.5')


EmbeddingsResponse(embedding=[0.5897687077522278, 0.401029109954834, -3.3032827377319336, -0.5248552560806274, 0.7503694295883179, 1.5193650722503662, -0.1255052536725998, 0.3970789909362793, 0.06761036068201065, -1.1095244884490967, 0.6922813653945923, 1.2783931493759155, 1.145704746246338, 1.0875918865203857, 0.2511359751224518, 0.293265700340271, 0.15194426476955414, -0.6336007118225098, -0.21012353897094727, -0.19572968780994415, -1.7954599857330322, -0.6283587217330933, 0.03834778070449829, -0.6681262254714966, 1.2611907720565796, 1.2775156497955322, -0.16066831350326538, -0.001951868529431522, -0.2972637116909027, -0.4810105264186859, 1.2045704126358032, -0.6384482979774475, -0.5407007932662964, -1.0363037586212158, 0.631036102771759, -1.2082654237747192, 0.6831682324409485, -0.05964048579335213, -0.1967417448759079, 0.1284455507993698, -0.014024718664586544, -0.5542796850204468, 0.3511347770690918, 0.0450400784611702, 0.5970171689987183, -0.9550229907035828, 0.5076606869697571, 

In [4]:
## Step 3 Splitting the document into chunks ##
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
documents=text_splitter.split_documents(text_documents)

In [5]:
## Step 4 Creating the vector store from the documents ##

vectorstore=FAISS.from_documents(documents,embeddings)
vectorstore.save_local("faiss_index")

In [6]:
from langchain_classic.chains import LLMChain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate



In [8]:
## 5 Querying the vector store from the date of the transaction
transaction_date= "24/06/18"

In [9]:
prompt_template = PromptTemplate.from_template(
    ("Summarize the amount for the transaction happened on {dateoftransaction}?")
)


In [10]:
docs=vectorstore.similarity_search("get me the transaction date on 24/06/18",k=3)
docs 

[Document(id='fa9eb6bf-8742-49d5-93b0-05571234e5b9', metadata={'source': 'sampletextfile'}, page_content='18/07/18          BO OKMYSHOW COM                       GURGAON                           606 .'),
 Document(id='bc8198eb-5858-470b-9d44-9c3ce897ddda', metadata={'source': 'sampletextfile'}, page_content='Date                   Details                        Merchant .City                  Amount'),
 Document(id='1ab30097-85a4-4b0a-93af-0857e7351ca9', metadata={'source': 'sampletextfile'}, page_content='01/07/18          AMAZON SELLER SERVICE S               MUMS AI                           286 .')]

In [11]:
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
docs=retriever.invoke({"question":"get me the transaction date on 24/06/18"})
docs

[Document(id='bc8198eb-5858-470b-9d44-9c3ce897ddda', metadata={'source': 'sampletextfile'}, page_content='Date                   Details                        Merchant .City                  Amount'),
 Document(id='fa9eb6bf-8742-49d5-93b0-05571234e5b9', metadata={'source': 'sampletextfile'}, page_content='18/07/18          BO OKMYSHOW COM                       GURGAON                           606 .'),
 Document(id='f020ccfd-b6a2-4151-9627-34a9ba8a02d2', metadata={'source': 'sampletextfile'}, page_content='465 . 00')]

In [12]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(
    model="gemma:2b",
    temperature=0.9,
    num_predict=256
)

In [13]:
input_text = "The meaning of life is "
response = model.invoke(input_text)
print(response)


The meaning of life is a question that has occupied philosophers and thinkers for centuries. It is a question that each individual must answer for themselves, as their own experiences, beliefs, and values shape their understanding of what life is about.

There is no single, correct answer to this question, and different perspectives exist. Some people find meaning in their relationships, others in their work, and still others in their pursuit of personal growth. Ultimately, the meaning of life is whatever brings you the most happiness, fulfillment, and meaning in your life.

Here are some different approaches to finding meaning:

* **Personal growth:** Focus on self-discovery, learning new skills, and challenging yourself to grow both intellectually and personally.
* **Relationships:** Build meaningful and fulfilling relationships with others who share your values and interests.
* **Service to others:** Find a way to contribute to your community and leave a positive impact on the world

In [14]:
## creating promt with the context from the vector store ## Retrieval Chain, Document chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)


In [15]:
from langchain_core.output_parsers import StrOutputParser

In [16]:
output_parser=StrOutputParser()

In [ ]:
## Creating the document chain ##
document_chain=create_stuff_documents_chain(llm=model, prompt=prompt, output_parser=output_parser)

In [23]:
## Creating the retrieval chain ##
from langchain_classic.chains.retrieval import create_retrieval_chain
retriever=vectorstore.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [25]:
retrieval_chain.invoke({"input":"get me the transaction date on 24/06/18"})

{'input': 'get me the transaction date on 24/06/18',
 'context': [Document(id='fa9eb6bf-8742-49d5-93b0-05571234e5b9', metadata={'source': 'sampletextfile'}, page_content='18/07/18          BO OKMYSHOW COM                       GURGAON                           606 .'),
  Document(id='bc8198eb-5858-470b-9d44-9c3ce897ddda', metadata={'source': 'sampletextfile'}, page_content='Date                   Details                        Merchant .City                  Amount'),
  Document(id='1ab30097-85a4-4b0a-93af-0857e7351ca9', metadata={'source': 'sampletextfile'}, page_content='01/07/18          AMAZON SELLER SERVICE S               MUMS AI                           286 .'),
  Document(id='4574d5c7-c025-4f52-b0a1-adfbfa133ebd', metadata={'source': 'sampletextfile'}, page_content='24/06/18          WITCO   ( INDIA) PVT LTD              CHENNAI                         2,049.00')],
 'answer': 'I am unable to access the context and cannot generate an answer based on the provided context.'}

In [ ]:
llm_chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
response= llm_chain.invoke({"dateoftransaction": transaction_date})

In [159]:
print(response)

{'city': 'India', 'text': 'The capital of India is New Delhi. It is the political and administrative center of the country.'}


In [ ]:
## creating promt with the context from the vector store


In [ ]:
document_chain = create_stuff_documents_chain(llm=model,prompt=prompt)
document_chain


ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['city']

In [63]:
## prompt template
from langchain import PromptTemplate   
template="""
You are a helpful assistant that helps people find information.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""  
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


ImportError: cannot import name 'PromptTemplate' from 'langchain' (/Users/parthasarathy.sivakasi/Library/CloudStorage/OneDrive-RackspaceInc/Projects/.venv/lib/python3.12/site-packages/langchain/__init__.py)

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics tracked on their usage graph.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [ ]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [ ]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023EF4D513F0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000023FB8A1DB10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023

In [ ]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'To set usage limits in LangSmith, navigate to **Settings -> Usage and Billing -> Usage configuration**. There, you will find a table at the bottom of the page that allows you to set usage limits per workspace. The two types of limits you can set are **total traces** and **extended retention traces**, each with an associated cost estimate.'

In [ ]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(page_content='use usage limits to prevent future overspend.LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estimate:Lets start by setting limits on our production usage, since that is where the majority of spend comes from.Setting a good total traces limit\u200bPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your assumptions before setting a limit.For example:Current Load: Our gen AI application is 